In [1]:
from config import Config
class cuyConfig(Config):
    """Configuration for training on the toy  dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "cuy_bw"

    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 2

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + balloon

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 500

    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9
class InferenceConfig(cuyConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    BACKBONE='resnet50'
config = InferenceConfig()

In [2]:
from model import MaskRCNN
#create a model in inference mode
model = MaskRCNN(mode="inference", model_dir='', config=config)
#load trained weights
model.load_weights("C:\\Users\\Lenovo\\Documents\\GitHub\\weight_mask_rcnn_cuy_0050.h5", by_name=True)
print("Loaded trained weights in the model")

C:\Users\Lenovo\Documents\GitHub\Weight-Mask-R-CNN\model.py:2899: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if os.name is 'nt':
Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Loaded trained weights in the model


In [3]:
import tensorflow as tf

keras_model = model.keras_model
converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
converter.allow_custom_ops = True
converter.experimental_new_converter = True
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
    tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]

converter.optimizations = [ tf.lite.Optimize.DEFAULT ]

tflite_model = converter.convert()

AttributeError: 'Model' object has no attribute '_get_save_spec'

In [3]:
#save the entire model
model.keras_model.save('cuy_bw_model_resnet50_1.h5')
print("Saved model in disk")

Saved model in disk


In [4]:
import model as modellib

In [5]:
import keras

In [6]:
#load entire model
model = keras.models.load_model('cuy_bw_model_resnet50_1.h5', custom_objects={ 'tf':modellib.tf, 'BatchNorm':modellib.BatchNorm, 'ProposalLayer':modellib.ProposalLayer, 'PyramidROIAlign':modellib.PyramidROIAlign, 'DetectionLayer':modellib.DetectionLayer}, compile=False)
print("Loaded model from disk")

TypeError: ('Keyword argument not understood:', 'config.BACKBONE')

In [7]:
for image_id in dataset.image_ids:
    print(image_id)
    image, image_meta, gt_class_id, gt_bbox, gt_mask, gt_weight =\
          modellib.load_image_gt_keypoints(dataset, config, image_id, use_mini_mask=False)
    info = dataset.image_info[image_id]
    print("image ID: {}.{} ({}) {}".format(info["source"], info["id"], image_id, info['weight']))
    # Run object detection
    results = model.detect_bodyweight([image], verbose=0)
    r = results[0]
    #print(r['rois'])
    print(r['bodyweight'])
#image_id = random.choice(dataset.image_ids)

#image, image_meta, gt_class_id, gt_bbox, gt_mask, gt_weight =\
#          modellib.load_image_gt_keypoints(dataset, config, image_id, use_mini_mask=False)
#info = dataset.image_info[image_id]
#print("image ID: {}.{} ({}) {}".format(info["source"], info["id"], image_id, info['weight']))

# Run object detection
#results = model.detect_bodyweight([image], verbose=0)

#log("gt_class_id", gt_class_id)
#log("gt_bbox", gt_bbox)
#log("gt_mask", gt_mask)
#log("gt_bodyweight", gt_weight)


    # Display results
#ax = get_ax(1)
#r = results[0]

#log("rois",r['rois'])
#log("class_ids",r['class_ids'])
#log("bodyweight",r['bodyweight'])
#log("masks",r['masks'])
#print(r['bodyweight'])
#visualize.display_instances_bw(image, r['rois'], r['masks'] ,r['bodyweight'], r['class_ids'], 
#                              dataset.class_names, r['scores'], ax=ax,
#                              title="Predictions")


NameError: name 'dataset' is not defined

In [6]:
#from sklearn.metrics import mean_squared_error
y_true, y_pred = [] , []
for image_id in dataset.image_ids:
    #print(image_id)
    image, image_meta, gt_class_id, gt_bbox, gt_mask, gt_weight =\
          modellib.load_image_gt_keypoints(dataset, config, image_id, use_mini_mask=False)
    info = dataset.image_info[image_id]
    #print("image ID: {}.{} ({}) {}".format(info["source"], info["id"], image_id, info['weight']))
    # Run object detection
    results = model.detect_bodyweight([image], verbose=0)
    r = results[0]
    if(np.size(r['bodyweight']) == 1):
        y_true.append(info['weight'])
        y_pred.append(r['bodyweight'][0][0])
    #print(r['bodyweight'])


In [7]:
from sklearn.metrics import r2_score
y_true_array = np.array(y_true)
y_pred_array = np.array(y_pred)
mse = (np.square(abs(y_true_array - y_pred_array))).mean()
rmse = np.sqrt(mse)
r2 = r2_score(y_true_array, y_pred_array)
print('MSE: %s' % mse)
print('RMSE: %s' % rmse)
print('R2: %s' % r2)

MSE: 56036.15194946707
RMSE: 236.71956393476876
R2: 0.39957950555867316


In [8]:
len(y_true_array)

1233